<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import urllib.request
import nltk
nltk.download('punkt')

urls = ['https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-ABC%20%EC%82%B4%EC%9D%B8%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EA%B7%B8%EB%A6%AC%EA%B3%A0%20%EC%95%84%EB%AC%B4%EB%8F%84%20%EC%97%86%EC%97%88%EB%8B%A4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%82%98%EC%9D%BC%EA%B0%95%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A7%8C%EC%B0%AC%ED%9A%8C%EC%9D%98%2013%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A9%94%EC%86%8C%ED%8F%AC%ED%83%80%EB%AF%B8%EC%95%84%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%AA%A9%EC%82%AC%EA%B4%80%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B2%99%EC%96%B4%EB%A6%AC%20%EB%AA%A9%EA%B2%A9%EC%9E%90.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B9%84%EB%B0%80%20%EC%84%9C%EB%A5%98%EB%A5%BC%20%EB%85%B8%EB%A0%A4%EB%9D%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%8A%A4%ED%8E%98%EC%9D%B8%EA%B6%A4%EC%A7%9D%EC%9D%98%20%EB%B9%84%EB%B0%80.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%84%ED%8C%8C%ED%8A%B8%EC%97%90%20%EB%82%98%ED%83%80%EB%82%9C%20%EC%9A%94%EC%A0%95.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%A0%ED%81%AC%EB%A1%9C%EC%9D%B4%EB%93%9C%20%EC%82%B4%EC%9D%B8%20%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%98%88%EA%B3%A0%20%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%A5%90%EB%8D%AB.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%BB%A4%ED%8A%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%81%AC%EB%A6%AC%EC%8A%A4%EB%A7%88%EC%8A%A4%20%ED%91%B8%EB%94%A9%EC%9D%98%20%EB%AA%A8%ED%97%98.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%91%B8%EB%A5%B8%EC%97%B4%EC%B0%A8%EC%9D%98%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%99%94%EC%9A%94%EC%9D%BC%20%ED%81%B4%EB%9F%BD%EC%9D%98%20%EC%82%B4%EC%9D%B8.txt']

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_korean_sentences(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40:
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
korean_sentences = collect_korean_sentences(urls)

In [ ]:
len(korean_sentences)

18036

In [ ]:
korean_sentences[100]

'그러나 당신이 받으셨다는 그 이상한 편지도 고려해 보고 싶습니다, 포아로 씨.'

In [13]:
import requests
import pandas as pd

client_id = "waXxTjNdGjzj8xtT3nbF" #1.에서 취득한 아이디 넣기
client_secret = "H6mejnkuYx"  #1. 에서 취득한 키 넣기

search_word = '코로나' #검색어
encode_type = 'json' #출력 방식 json 또는 xml
max_display = 100 #출력 뉴스 수
sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
start = 1 # 출력 위치

url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

#헤더에 아이디와 키 정보 넣기
headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret':client_secret
           }

#HTTP요청 보내기
r = requests.get(url, headers=headers)
#요청 결과 보기 200 이면 정상적으로 요청 완료
print(r)

<Response [200]>


In [17]:
df = pd.DataFrame(r.json()['items'])

In [18]:
def clean_html(x):
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

df['title'] = df['title'].apply(lambda x: clean_html(x))
df['description'] = df['description'].apply(lambda x: clean_html(x))

In [19]:
df

,title,originallink,link,description,pubDate
0,"아산시, 코로나19 확진자 2명(#387~#388) 추가 발생",http://www.newstnt.com/news/articleView.html?i...,http://www.newstnt.com/news/articleView.html?i...,코로나19 / 뉴스티앤티 DB 아산시(시장 오세현)는 10일 코로나19 확진자 2명...,"Wed, 10 Mar 2021 12:27:00 +0900"
1,한수진·조성연 연주 제1회 부산클래식음악제는 순항 중···,http://kor.theasian.asia/archives/287615,http://kor.theasian.asia/archives/287615,코로나 방역으로 인한 거리두기 탓에 880석의 금빛누리홀이나 394석의 은빛샘홀 공...,"Wed, 10 Mar 2021 12:27:00 +0900"
2,"해수부 문성혁 장관, 수협 방문 간담회 열어",http://www.thepublic.kr/news/newsview.php?ncod...,http://www.thepublic.kr/news/newsview.php?ncod...,문성혁 장관은 이번 간담회에서 포스트 코로나 시대에 대비한 비대면 시대 맞춤 수산간...,"Wed, 10 Mar 2021 12:26:00 +0900"
3,美 접종자 지침에 논란···백신 맞으면 마스크 벗어도 될까,https://www.sedaily.com/NewsView/22JR5UD4UM,https://news.naver.com/main/read.nhn?mode=LSD&...,미국 질병통제예방센터(CDC)는 지난 8일(현지시간) '코로나19 백신 접종자들끼리...,"Wed, 10 Mar 2021 12:26:00 +0900"
4,"방탄소년단 '다이너마이트' 뮤직비디오, 9억뷰 달성",http://www.spotvnews.co.kr/?mod=news&act=artic...,https://news.naver.com/main/read.nhn?mode=LSD&...,'다이너마이트'는 코로나19로 모두가 힘든 시기에 활력과 희망의 메시지를 전하고자 ...,"Wed, 10 Mar 2021 12:26:00 +0900"
...,...,...,...,...,...
95,"전남농업기술원, 올해 농업분야 신기술 보급 393억 투입",https://www.asiatoday.co.kr/view.php?key=20210...,https://www.asiatoday.co.kr/view.php?key=20210...,계획이다.코로나19 상황 장기화에 따른 국제 곡물 가격 급등으로 식량 안정생산을 위...,"Wed, 10 Mar 2021 12:19:00 +0900"
96,"BGF리테일 CU, 국내 편의점업계 최초로 할랄 인증 간편식 내놔",http://www.businesspost.co.kr/BP?command=artic...,http://www.businesspost.co.kr/BP?command=artic...,김동률 BGF리테일 간편식품팀 MD(상품기획자)는 코로나19로 이색 상품에 대한 고...,"Wed, 10 Mar 2021 12:19:00 +0900"
97,[올댓차이나] 중국 2월 생산자물가 1.7%↑ 소비자물가 0.2%↓(종합),http://www.newsis.com/view/?id=NISX20210310_00...,https://news.naver.com/main/read.nhn?mode=LSD&...,수출 급증에 기인한 바가 컸으며 중국 경제가 코로나19 충격에서 벗어나 견실하게 성...,"Wed, 10 Mar 2021 12:19:00 +0900"
98,중국 '백신여권' 국제건강증명 도입 여행 기대감 쑥~ 테마주 주가 훨훨,http://www.newspim.com/news/view/20210310000584,http://www.newspim.com/news/view/20210310000584,중국은 자국이 도입한 국제여행건강증명서 제도가 향후 코로나19 상시화 국면에서 여행...,"Wed, 10 Mar 2021 12:19:00 +0900"


In [20]:
import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool
import pandas as pd

In [36]:
headers = {'User-Agent':'Chrome/66.0.3359.181'}
def get_bs_obj(url):
    result = requests.get(url,headers=headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return bs_obj


def get_content(link):
    bs_obj = get_bs_obj(link)
    main_news = []

    #print(bs_obj)
    try:
        title = bs_obj.find("h3",{"id":"articleTitle"}).text
        body = bs_obj.find("div",{"class":"_article_body_contents"}).get_text("\n").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}","")
        aid = bs_obj.find("div",{"class":"article_header"}).find("img")["title"]
        main_news = [title,clean_text(body)]
    except:
        a=1
    return main_news


In [39]:
for url in df['link']:
    if url.startswith('https://news.naver.com/'):
        print(get_content(url))

['美 접종자 지침에 논란···백신 맞으면 마스크 벗어도 될까', '美CDC 접종자 마스크 지침 완화···韓질병청은 착용해야 백신 맞아도 100% 항체 형성 아냐···경증·무증상 전파 가능 변이에 백신 효과 의문···집단면역 전엔 마스크 착용 불가피 지난 8일 오후 광주 동구 전남대병원에서 신종 코로나바이러스 감염증(코로나19) 아스트로제네카(AZ) 백신이 의료진 접종을 위해 준비돼 있다. /연합뉴스 [서울경제] 미국에서 백신 접종자를 대상으로 한 마스크 착용 지침이 완화되면서 백신 접종자의 마스크 착용 의무에 대한 논란이 일고 있다. 미국 질병통제예방센터(CDC)는 지난 8일(현지시간) 코로나19 백신 접종자들끼리는 마스크를 쓰지 않아도 되고 사회적 거리두기도 불필요하다고 발표했다. CDC는 완전한 백신 접종자(백신별로 규정된 마지막 회차분 백신을 맞은 때로부터 2주가 지난 사람)는 다른 백신 접종자 혹은 중증을 앓을 위험성이 낮고 한 가족 구성원인 비접종자와 실내에서 만날 때 마스크를 쓰거나 사회적 거리두기를 하지 않아도 된다고 권고했다. 완전한 백신 접종자가 전체 인구의 약 9%(약 3100만명)에 도달한 시점에 마스크 지침을 완화한 미국과 달리 백신 접종을 막 시작한 한국 방역당국은 기존의 강력한 마스크 지침을 유지할 방침이다. 정은경 질병관리청장은 8일 국회 긴급현안질문에서 백신 접종이 시작된다 하더라도 완전히 코로나19가 종식되는 건 아니기 때문에 좀 더 안전해질 때까지 마스크 착용 역학조사 및 방역 대응은 같이 진행돼야 한다고 말했다. 미국 질병통제예방센터(CDC)가 지난 8일(현지시간) 새 코로나19 방역지침을 발표했다. /CDC 홈페이지 캡처 ◇ 100% 예방효과 백신 없어···시간 지날수록 감염 위험성↑ 이론상 백신 접종자에게 마스크 착용이 필요없다고 단언하기 위해서는 해당 접종자가 더 이상 코로나19에 감염되거나 다른 사람에게 바이러스를 전파할 위험이 없다는 점이 증명돼야 한다. 백신 접종으로 항체가 형성되는 등의 예방효과가 100% 발생하

KeyboardInterrupt: ignored